# The algorithm as it stands

Take two catalogues as input:
- S3M is a simulated catalogue that contains a representative population of the solar system
- MPCOrb contains real data from actual detections

The goal is to combine the two to make a hybrid catalogue.

In order to do this we will:
1. Split catalogues into bins of absolute magnitude and for each bin
    1. Create a K-D Tree for the matching objects in the simulated catalogue (`tree`)
    2. Create a set of all objects that have already been assigned (`assigned`)
    3. For each object in the mpcorb catalogue with matching magnitudes:
        1. Set `k=100`
        2. Query the `tree` to find the nearest `k` objects in position within a distance `d_max`, save as `neighbours` and save length of `neighbours` as `count_nearby`
        3. Remove any objects that are already in `assigned`
        4. While `len(neighbours) < 1` and `count_nearby > 0` and `k < 500`
            1. `k += 100`
            2. Go back to step b
        5. If `len(neighbours) >= 1` then
            1. Find the object in `neighbours` with the closest velocity
            2. Save the pairing and add the object to `assigned`
        6. Otherwise
            1. Make a note that the real object has no match (so we will add it directly)
3. Output the list of pairings

In [362]:
from scipy.spatial import KDTree, cKDTree
import numpy as np
import pandas as pd

## Read in the data

In [9]:
s3m = pd.read_hdf("catalogues/s3m_propagated.h5", key="df")

In [13]:
mpcorb = pd.read_hdf("catalogues/mpcorb_propagated.h5", key="df")

In [97]:
mpcorb_xyz = np.array([mpcorb["x"].values, mpcorb["y"].values, mpcorb["z"].values]).T
s3m_xyz = np.array([s3m["x"].values, s3m["y"].values, s3m["z"].values]).T

## Split into absolute magnitude bins

In [191]:
sorted_H = np.sort(s3m.H.values)

In [201]:
block_size = 5000
blocks = np.concatenate((np.arange(0, len(s3m.H.values), block_size), [len(s3m.H.values) - 1]))

In [327]:
s3m.H.values.min()

-1.757

In [328]:
s3m.H.values.max()

26.69

In [363]:
%%timeit -n 1 -r 1

magnitudes = np.arange(-2, 28)
for i in range(len(magnitudes) - 1):
    i = 14
    s3m_mag_mask = np.logical_and(s3m.H >= magnitudes[i], s3m.H < magnitudes[i + 1])
    tree = cKDTree(s3m_xyz[s3m_mag_mask])
    
    mpcorb_mag_mask = np.logical_and(mpcorb.H >= magnitudes[i], mpcorb.H <= magnitudes[i + 1])
    real_objects = mpcorb_xyz[mpcorb_mag_mask]
    print(len(s3m.H[s3m_mag_mask]), len(real_objects))
    
    taken = []
    x = 0
    
    for obj in real_objects:
        k = 100
        distances, inds = tree.query(obj, k=k, distance_upper_bound=0.1)
        distances, inds = distances[np.isfinite(distances)], inds[np.isfinite(distances)]
        unmatched_options = np.setdiff1d(inds, taken, assume_unique=True)
        while len(unmatched_options) < 1 and len(distances) > 0 and k < 500:
            k += 100
            distances, inds = tree.query(obj, k=k, distance_upper_bound=0.1)
            distances, inds = distances[np.isfinite(distances)], inds[np.isfinite(distances)]
            unmatched_options = np.setdiff1d(inds, taken, assume_unique=True)

        if len(unmatched_options) > 0:
            match = unmatched_options[0]
            taken.append(match)
        else:
            x += 1
            
    break

8404 5364
1.8 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [323]:
other_tree = KDTree(mpcorb_xyz[mpcorb_mag_mask])

In [282]:
taken = []
x = 0



In [283]:
mpcorb.H[mpcorb_mag_mask]

0          3.54
1          4.22
2          5.28
3          3.40
4          6.99
           ... 
1123446    6.90
1124470    6.70
1126135    6.90
1126837    6.10
1126850    5.70
Name: H, Length: 1402, dtype: float64

## Get the positions

In [29]:
mpcorb_xyz = np.array([mpcorb["x"].values, mpcorb["y"].values, mpcorb["z"].values]).T
s3m_xyz = np.array([s3m["x"].values, s3m["y"].values, s3m["z"].values]).T

## Build the K-D Tree for S3m

In [48]:
%time s3m_tree = KDTree(s3m_xyz)

CPU times: user 2min 4s, sys: 11.1 s, total: 2min 16s
Wall time: 2min 16s


In [81]:
np.histogram(s3m.H, bins=10)

(array([     10,     117,    6720,   43993,    7538,  153727, 1341057,
        5911117, 6539988,  385541]),
 array([-1.757 ,  1.0877,  3.9324,  6.7771,  9.6218, 12.4665, 15.3112,
        18.1559, 21.0006, 23.8453, 26.69  ]))

In [47]:
%time mpcorb_tree.query(x=np.random.uniform(0, 10, size=(10000, 3)))

CPU times: user 4.14 s, sys: 142 ms, total: 4.29 s
Wall time: 4.28 s


(array([1.64632104, 1.38976017, 2.62848638, ..., 1.31153415, 0.11273276,
        0.86779311]),
 array([1019396,  874547,  584777, ...,  620807,  673756,  481031]))

In [147]:
import astropy.units as u
import astropy.constants as const

T = 2.7 * u.K

a = const.sigma_sb * 4 / const.c
ed = a * T**4
ed.to(u.eV / u.cm**3)

<Quantity 0.2509549 eV / cm3>

In [170]:
u.G.decompose()

Unit("0.0001 kg / (A s2)")

In [172]:
B = 10e-6 * u.cm**(-1/2) * u.g**(1/2) * u.s**(-1)
ed = 1 / (8 * np.pi) * B**2
ed.to(u.eV / u.cm**3)

<Quantity 2.48341755 eV / cm3>